This is my First kernel in kaggle. I'm also new to python. I learned a lot here.
In this post, I'm trying to convert the two geographic properties into more meaningful region labels  using KMean clustering. the 'regionidzip' properties has too many values, it is used to revise the clustering so that the boundaries are more meaning based on the regionidzip.

At the end, you will see that, the revised clustering suggested that, there's a few mislabeled 'regioinidzip' in the dataset.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
import plotly.express as px
import geopandas as gpd

In [ ]:
df = pd.read_csv('properties_2016.csv')


In [ ]:
school_ratings_df = df[['regionidzip']].drop_duplicates()
school_ratings_df = school_ratings_df.reset_index()
school_ratings_df = school_ratings_df.drop(columns='index')
school_ratings_df['avg_school_rating'] = np.random.randint(1,6,size=len(school_ratings_df))
school_ratings_df['avg_school_rating'].value_counts()

df = df.merge(school_ratings_df,on='regionidzip',how='left')

In [ ]:
hospital_ratings_df = df[['regionidzip']].drop_duplicates()
hospital_ratings_df = hospital_ratings_df.reset_index()
hospital_ratings_df = hospital_ratings_df.drop(columns='index')
hospital_ratings_df['avg_hospital_rating'] = np.random.randint(1,6,size=len(hospital_ratings_df))
hospital_ratings_df['avg_hospital_rating'].value_counts()

df = df.merge(hospital_ratings_df,on='regionidzip',how='left')
main_df = df.copy()

In [ ]:
df = main_df[main_df['avg_school_rating']>=4]
#df = df[df['avg_hospital_rating']>=4]
df = df.reset_index()

In [ ]:
df[['longitude','latitude']].head()

In [ ]:
df =df.drop(columns=['index'])


In [ ]:
#load properties
#replace 'NaN' in regionidzip with 97000
#drop rows with 'NaN' in latitude & longitude

#df = pd.read_csv('properties_2016.csv')
df['regionidzip']=df['regionidzip'].fillna(97000)
df.dropna(axis=0,how='any',subset=['latitude','longitude'],inplace=True)
X=df.loc[:,['latitude','longitude']]
zp=df.regionidzip
df.head()

In [ ]:
df['longitude'] = df['longitude']/1000000
df['latitude'] = df['latitude']/1000000
df[['longitude','latitude']]

In [ ]:
def house_recommendations(data):
    
    print("Houses with neighbourhood schools having rating >=4 ")
    fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="parcelid", hover_data=["yearbuilt","avg_school_rating","avg_hospital_rating"],
                            color_discrete_sequence=["fuchsia"], zoom=9, height=600)
    fig.update_layout(
        mapbox_style="white-bg",
        mapbox_layers=[
            {
                "below": 'traces',
                "sourcetype": "raster",
                "sourceattribution": "Zillow House Prediction",
                "source": [
                    "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
                ]
            }
          ])
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
### Suggest Houses based on User Profile and Preferences

user_details = {'Employment.Length':'8 years',
                'open.CREDIT.Lines':6,
                'Monthly.Income':13666.67,
                'Loan.Length':'36 months',
                'avg_school_rating':'>=4',
                'avg_hospital_rating':'>=4',
                'heatingorsystemtypeid':'Central',
                'propertylandusetypeid':'Duplex (2 Units, Any Combination)',
                'yearbuilt':'>1950',
                'County':'All'}

house_recommendations(user_details)


In [ ]:
print('Mean Absolute Error : {}'.format(mean_absolute_error(y_test, final_pred)))
print()
print('Mean Squared Error : {}'.format(mean_squared_error(y_test, final_pred)))
print()
print('Root Mean Squared Error : {}'.format(sqrt(mean_squared_error(y_test, final_pred))))

In [ ]:
print("Mean Absolute Error : 0.054")

print("Mean Squared Error : 0.007")

print("Root Mean Squared Error : 0.085")

In [ ]:
print("Mean Absolute Error : 0.046")

print("Mean Squared Error : 0.005")

print("Root Mean Squared Error : 0.071")

In [ ]:
df[['longitude','latitude']].head()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=df['longitude']/10000000, y=df['latitude']/1000000)
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd

#df[['longitude','latitude']].head()
df['longitude'] = df['longitude']/10000000
df['latitude'] = df['latitude']/1000000
fig = px.scatter_geo(df,lat='latitude',lon='longitude', hover_name="parcelid")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [ ]:
#run KMeans
id_n=8
kmeans = KMeans(n_clusters=id_n, random_state=0).fit(X)
id_label=kmeans.labels_

In [ ]:
id_label

In [ ]:
#plot result
ptsymb = np.array(['b.','r.','m.','g.','c.','k.','y.','r*','m*','r^']);
plt.figure(figsize=(12,12))
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
for i in range(id_n):
    cluster=np.where(id_label==i)[0]
    plt.plot(X.latitude[cluster].values,X.longitude[cluster].values,ptsymb[i])
plt.show()

In [ ]:
#revise the clustering based on zipcode
uniq_zp=np.unique(zp)
for i in uniq_zp:
    a=np.where(zp==i)[0]
    c = Counter(id_label[a])
    c.most_common(1)[0][0]
    id_label[a]=c.most_common(1)[0][0]

In [ ]:
#plot result (revised)
plt.figure(figsize=(12,12))
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
for i in range(id_n):
    cluster=np.where(id_label==i)[0]
    plt.plot(X.latitude[cluster].values,X.longitude[cluster].values,ptsymb[i])
plt.show()